In [4]:
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
import seaborn as sns

sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("vlag"))

In [5]:
from datetime import datetime, timedelta

# BDay is business day, not birthday...
from pandas.tseries.offsets import BDay
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [6]:
import yfinance as yf

# set the end date to today
end_date = datetime.today()
# set the start date to 5 years ago
start_date = end_date - timedelta(days=5 * 365)
print(start_date)

2019-07-04 14:29:32.120561


In [79]:
# list of tickers
tickers = [
    "^NDX",
    "QQQ",  # Invesco QQQ Trust
    "SHV",  # iShares Short Treasury Bond ETF
    "TLT",  # iShares 20+ Year Treasury Bond ETF
]
market = ["^GSPC"]

close_df = pd.DataFrame()
for _t in tickers:
    close_df[_t] = yf.download(_t, start=start_date, end=end_date)["Close"]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [80]:
close_df.head()

,^NDX,QQQ,SHV,TLT
Date,,,,
2019-07-01,7768.140137,189.259995,110.449997,132.240005
2019-07-02,7799.819824,190.009995,110.470001,133.259995
2019-07-03,7857.689941,191.440002,110.500000,134.210007
2019-07-05,7841.299805,191.050003,110.470001,132.429993
2019-07-08,7785.790039,189.710007,110.489998,132.610001


#### Nasdaq 100 composition

In [60]:
import requests

headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
}
res = requests.get(
    "https://api.nasdaq.com/api/quote/list-type/nasdaq100", headers=headers
)
print(res.status_code)
main_data = res.json()["data"]["data"]["rows"]

200


In [81]:
ndx_comp = pd.DataFrame(main_data)
ndx_comp["marketCap"] = (
    ndx_comp["marketCap"].str.replace(",", "").astype("float") / 10**9
)
ndx_comp.sort_values("marketCap", ascending=False)

# ndx_comp[ndx_comp["symbol"] == "TSLA"]

,symbol,sector,companyName,marketCap,lastSalePrice,netChange,percentageChange,deltaIndicator
13,MSFT,,Microsoft Corporation Common Stock,3321.869075,$446.95,$5.90,-1.3%,down
0,AAPL,,Apple Inc. Common Stock,3229.664351,$210.62,$3.48,-1.63%,down
39,NVDA,,NVIDIA Corporation Common Stock,3038.879167,$123.54,$0.45,-0.36%,down
80,GOOG,,Alphabet Inc. Class C Capital Stock,2266.704360,$183.42,$3.44,-1.84%,down
63,GOOGL,,Alphabet Inc. Class A Common Stock,2251.009700,$182.15,$3.26,-1.76%,down
...,...,...,...,...,...,...,...,...
87,MDB,,"MongoDB, Inc. Class A Common Stock",18.334740,$249.96,$5.81,+2.38%,up
97,WBD,,"Warner Bros. Discovery, Inc. Series A Common S...",18.230332,$7.44,$0.09,+1.22%,up
44,ILMN,,"Illumina, Inc. Common Stock",16.627734,$104.38,$2.08,-1.95%,down
60,WBA,,"Walgreens Boots Alliance, Inc. Common Stock",10.434518,$12.095,$0.095,-0.78%,down


#### interest rates

In [55]:
%reload_ext autoreload
%autoreload 2

from interest_rate_fetcher import get_interest_rate

daily_spot_rate = get_interest_rate(sample_start=start_date, maturity="DGS6MO")

In [54]:
daily_spot_rate.head()

,6.0
2019-07-01,0.0210
2019-07-02,0.0209
2019-07-03,0.0208
2019-07-05,0.0214
2019-07-08,0.0214


### Modern Portfolio Theory (MPT)